In [2]:
import os
import sys
import csv
import time
import string
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [3]:
#list_next_page = []

def get_nextpage(url):

    page1 = requests.get(url)
    soup1 = BeautifulSoup(page1.text, "html.parser")
    btn = soup1.find('div', class_='pagination')
    btn_next_li = btn.find_all('li')
    btn_next = btn.find_all('a', class_='prevnext')

    # 目前頁面網址
    page_href = btn_next[1]['href']

    # 下一頁網址
    next_page_url = 'https://www.8notes.com' + page_href
    url = next_page_url

    try:
        if next_page_url == 'https://www.8notes.comjavascript:;':
            pass
        else:
            list_next_page.append(url)
            get_nextpage(url=next_page_url)
    except:
        pass
    
    return list_next_page

In [4]:
#list_title = []
#list_title_url = []
#df_midiurl = pd.DataFrame()

def get_title(df_miditype):
    
    # 迴圈抓取下一頁(利用list_next_page陣列的每一頁網址抓取)
    for i in range(len(list_next_page)):
        v = list_next_page[i]
        page2 = requests.get(v)
        soup2 = BeautifulSoup(page2.text, "html.parser")

        title = soup2.findAll("span", {"class": "listbox redtitle"})
        title_url = soup2.findAll("a", {"class": "listboxrow"})

        for t, u in zip(title, title_url):
            list_title.append(t.text)

            # 比對字串：url如果有包含'/scores/'就保留，否則為空白
            if str.find(u['href'],'/scores/') != -1:
                list_title_url.append(u['href'])
            else:
                list_title_url.append('')
    
    # 曲目名稱和曲目頁面網址放入dataframe中
    df_midiurl['Title'] = pd.Series(list_title)
    df_midiurl['Url'] = pd.Series(list_title_url)
    
    return df_midiurl

In [15]:
# 取得每一頁的MIDI檔案並下載在相對應的資料夾(folder_path)
# 加入例外狀況 (含有/scores/字串，但ftype=ext)

def get_eachpage_midi(df_midiurl):

    for i in range(len(df_midiurl)):

        # 把曲目頁面網址用完整並導向MIDI檔案格式
        x = df_midiurl['Url'][i]

        # URL是否為空
        if x != '':
            midi_url = 'https://www.8notes.com' + x + '?ftype=midi'
            #print(midi_url)
            list_midi_url.append(midi_url)

            page3 = requests.get(midi_url)
            soup3 = BeautifulSoup(page3.text, "html.parser")

            # MIDI下載按鈕的連結
            results = soup3.find("a", {"class": "transp_alphabet"}).attrs["href"]
            #print(results)
            
            # 判斷midi下載網址是否能下載，若回傳的href為/school/midi，則不執行midi下載指令
            if results == '/school/midi/':
                download_url = 'https://www.8notes.com' + results
                list_download_url.append(download_url)
            else:
                download_url = 'https://www.8notes.com' + results
                list_download_url.append(download_url)
                #print('Download:', download_url)

                filename = download_url.split('/')[::-1][0]
                #print(filename)

                mid_file = requests.get(download_url, stream=True)

                # MIDI開始下載
                with open(folder_path + '/' + filename, 'wb') as saveMidFile:
                    saveMidFile.write(mid_file.content)
                    #print('Downloaded {} successfully.'.format(filename))
    
        # 如果URL為空，就append空白進去dataframe
        else:
            list_midi_url.append('')
            list_download_url.append('')
            pass
    
    # 曲目midi頁面和midi下載路徑新增放入dataframe中    
    df_midiurl['MidiUrl'] = pd.Series(list_midi_url)
    df_midiurl['DownloadUrl'] = pd.Series(list_download_url)
    
    #return list_midi_url, list_download_url, df_midiurl
    return df_midiurl

In [17]:
df_miditype = pd.read_csv(r'csv/df_miditype.csv')
list_next_page = []
list_title = []
list_title_url = []
list_midi_url = []
list_download_url = []
df_midiurl = pd.DataFrame()

#for count in range (1,len(df_miditype)):
for count in range(15, 16):
    print('Round:', count)
    
    type = df_miditype['Type'][count]
    url = df_miditype['Url'][count]
    folder_path = 'Midi/' + str(count).zfill(2) + '_' + str(type)
    os.mkdir(folder_path)
    print('Create Folder [ ' + folder_path + ' ] Finish!')

    list_next_page = get_nextpage(url)
    list_next_page.insert(0, url)
    #print(list_next_page)
    
    df_midiurl = get_title(df_miditype)
    a = 'csv/' + str(count).zfill(2) + '_' + str(type) + '_df_midiurl.csv'
    print('Saved in ' + str(a))
    df_midiurl.to_csv(a, index=False)

    get_eachpage_midi(df_midiurl)
    b = 'csv/' + str(count).zfill(2) + '_' + str(type) + '_df_midiurl_final.csv'
    print('Saved in ' + str(b))
    df_midiurl.to_csv(b, index=False)

    DIR = folder_path  #要統計的資料夾
    file_count = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
    print(str(file_count) + ' files in [ ' + folder_path + ' ]')
    print('-------------------------------------------------------------------')

Round: 15
Create Folder [ Midi/15_Advanced Piano ] Finish!
Saved in csv/15_Advanced Piano_df_midiurl.csv
Saved in csv/15_Advanced Piano_df_midiurl_final.csv
279 files in [ Midi/15_Advanced Piano ]
-------------------------------------------------------------------
